In [39]:
from urllib.request import urlopen

In [40]:
from urllib.request import Request

In [36]:
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

In [37]:
HEADERS

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

In [6]:
from bs4 import BeautifulSoup

In [62]:
KARTHUR = 'https://www.kingarthurflour.com/'
KARTHUR_SEARCH='https://search.kingarthurflour.com/search?p=Q&lbc=kingarthurflour&uid=16333142&ts=custom&w=bread&method=and&view=grid&af=recipe_cat1%3abread%20type%3arecipes&cnt=900&isort=title&srt=0'

In [87]:
def get_search_results(url):
    req = Request(url=url, headers=HEADERS)
    with urlopen(req) as response:
        x = response.read()
    xs = BeautifulSoup(x, 'lxml')
    return xs

In [79]:
name_to_page = {}

In [68]:
def process_h5(t):
    # Get the <a href> tag
    atags = [_ for _ in t.children if _.name == 'a']
    if len(atags) != 1:
        print(f'Warning, found {len(atags)} atags! expecting 1')
    for a in atags:
        try:
            bread_name = a.text.strip()
            print(f'processing {bread_name}')
            page_url = a.attrs['title']
            if bread_name in name_to_page:
                print(f'warning! replacing {name_to_page[bread_name]} with {page_url}')
            name_to_page[bread_name] = page_url
        except:
            print ("Skipping {str(a)} because couldn't find title or valid text!")

In [77]:
xs = list()

In [78]:
for i in range(3):
    xs.append(get_search_results(url=KARTHUR_SEARCH + str(i*300)))
    

/Users/andrewho/.pyenv/versions/breadventure/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/andrewho/.pyenv/versions/3.6.5/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [80]:
counts = list()
for i in range(3):
    count = 0
    for _ in xs[i].find_all('h5'):
        count += 1
        process_h5(_)
    counts.append(count)

processing 9-Grain Bread
processing 10-Grain Orange Bread
processing 100% Whole Wheat Banana Bread
processing 100% Whole Wheat Bread for the Bread Machine
processing 100% Whole Wheat Cinnamon Swirl Bread
processing 100% Whole Wheat Pain de Mie
processing 100% Whole Wheat Sandwich Bread
processing 100% Whole Wheat Zucchini Chocolate Chip Bread
processing A Dozen Simple Bagels
processing A Simple, Rustic Loaf
processing A Smaller 100% Whole Wheat Pain de Mie
processing A Smaller Pain de Mie
processing Absolutely No-Knead Crusty-Chewy Bread
processing Alexandra's Crescents
processing All-American Ham and Cheese Roll
processing Almond Cracked Wheat Bread
processing Almond Flour Zucchini Bread
processing Almond Sweet Bread Kit
processing American Irish Soda Bread
processing American-Style Panettone
processing Amish Dinner Rolls
processing Ancient Grains Bread
processing Angel Biscuits
processing Anise-Chocolate Flatbread
processing Apple Cheddar Chop Bread
processing Apple Cheese Loaf
proce

In [81]:
counts

[306, 306, 287]

In [83]:
## Save the dictionary of urls
import pickle

In [85]:
with open('name_to_page.pkl', 'wb') as f:
    pickle.dump(file=f, obj=name_to_page)

In [88]:
xs = get_search_results('https://www.kingarthurflour.com/recipes/gluten-free-golden-millet-cornbread-recipe')

/Users/andrewho/.pyenv/versions/breadventure/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/andrewho/.pyenv/versions/3.6.5/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [ ]:
# https://www.kingarthurflour.com/recipes/PrintRecipe?RID=6617&radio=3

In [107]:
get_recipe_id('https://www.kingarthurflour.com/recipes/gluten-free-golden-millet-cornbread-recipe')

found 6617 for https://www.kingarthurflour.com/recipes/gluten-free-golden-millet-cornbread-recipe


/Users/andrewho/.pyenv/versions/breadventure/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/andrewho/.pyenv/versions/3.6.5/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [118]:
import time
time.sleep(0.5)

In [130]:
import typing
class Recipe(object):
    def __init__(self):
        self.url = ''
        self.rid = None
        self.title = ''
        self.text = ''

In [131]:
def get_recipe_id(recipe: Recipe):
    """ Parse the recipe page for the recipe id so we can get print friendly version"""
    xs = get_search_results(recipe.url)
    tags = xs.findAll("input", {"data-serverclass": "recipeId"})
    rid = tags[0].attrs['value']
    for tag in tags:
        if rid != tag.attrs['value']:
            print(f"Warning! mismatched tags: {(rid, tag.attrs['value'])})")
    if url in recipe_url_to_id:
        print(f'Warning! replacing id {recipe_url_to_id[url]} with {rid}')
    else:
        print(f'found id {rid} for {recipe.title}')
    return int(rid)

In [132]:
recipes = list()

In [133]:
for title, url in name_to_page.items():
    print(f'working on {title}')
    r = Recipe()
    r.url = url
    r.title = title
    r.rid = get_recipe_id(r)
    recipes.append(r)
    time.sleep(0.5)  # Don't DOS the server

working on 9-Grain Bread


/Users/andrewho/.pyenv/versions/breadventure/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/andrewho/.pyenv/versions/3.6.5/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


found id 4889 for 9-Grain Bread
working on 10-Grain Orange Bread
found id 7791 for 10-Grain Orange Bread
working on 100% Whole Wheat Banana Bread
found id 2053 for 100% Whole Wheat Banana Bread
working on 100% Whole Wheat Bread for the Bread Machine
found id 5997 for 100% Whole Wheat Bread for the Bread Machine
working on 100% Whole Wheat Cinnamon Swirl Bread
found id 4217 for 100% Whole Wheat Cinnamon Swirl Bread
working on 100% Whole Wheat Pain de Mie
found id 7199 for 100% Whole Wheat Pain de Mie
working on 100% Whole Wheat Sandwich Bread
found id 27 for 100% Whole Wheat Sandwich Bread
working on 100% Whole Wheat Zucchini Chocolate Chip Bread
found id 3829 for 100% Whole Wheat Zucchini Chocolate Chip Bread
working on A Dozen Simple Bagels
found id 6589 for A Dozen Simple Bagels
working on A Simple, Rustic Loaf
found id 6275 for A Simple, Rustic Loaf
working on A Smaller 100% Whole Wheat Pain de Mie
found id 1811 for A Smaller 100% Whole Wheat Pain de Mie
working on A Smaller Pain de

found id 3931 for Buttery Sourdough Buns
working on Buttery Sourdough Sandwich Biscuits
found id 5647 for Buttery Sourdough Sandwich Biscuits
working on Cabot Cheddar Soda Bread
found id 1857 for Cabot Cheddar Soda Bread
working on Calzone
found id 6653 for Calzone
working on Canapé Pumpernickel Bread
found id 2093 for Canapé Pumpernickel Bread
working on Candied Cherry and Almond Biscotti
found id 5165 for Candied Cherry and Almond Biscotti
working on Caramel Apple Bread Pudding
found id 9180 for Caramel Apple Bread Pudding
working on Caramel-Apple Cinnamon Buns
found id 4983 for Caramel-Apple Cinnamon Buns
working on Caramel Bread Pudding with Coconut and Banana
found id 4675 for Caramel Bread Pudding with Coconut and Banana
working on Caramel-Nut Cinnamon Buns
found id 2095 for Caramel-Nut Cinnamon Buns
working on Caraway Rye Bread
found id 4807 for Caraway Rye Bread
working on Challah
found id 6943 for Challah
working on Cheddar Cheese Bread
found id 8099 for Cheddar Cheese Bread
w

working on Cumin Cornmeal Buns with Green Chili-Cheese Butter
found id 8493 for Cumin Cornmeal Buns with Green Chili-Cheese Butter
working on Czech Kolaches
found id 8987 for Czech Kolaches
working on Daily Bread
found id 6719 for Daily Bread
working on Dainty Tea-Sandwich Bread
found id 7239 for Dainty Tea-Sandwich Bread
working on Dairy Rolls
found id 8191 for Dairy Rolls
working on Daktyla
found id 7523 for Daktyla
working on Danish Cheese Buns
found id 1088 for Danish Cheese Buns
working on Danish Cream Rolls
found id 8979 for Danish Cream Rolls
working on Danish Pastry
found id 4783 for Danish Pastry
working on Dark Chocolate Bread Pudding
found id 6751 for Dark Chocolate Bread Pudding
working on Dark Currant & Nut Loaf
found id 8845 for Dark Currant & Nut Loaf
working on Dark Ginger Quickbread
found id 7275 for Dark Ginger Quickbread
working on Dark Pumpernickel-Onion Loaf
found id 6835 for Dark Pumpernickel-Onion Loaf
working on Dark & Soft Restaurant Dinner Rolls
found id 6467 

...

working on King Arthur's Classic White Sandwich Bread
found id 1446 for King Arthur's Classic White Sandwich Bread
working on King Arthur's Egg Braid
found id 8407 for King Arthur's Egg Braid
working on Lanesville Nisu
found id 7823 for Lanesville Nisu
working on Large-Batch Hot Dog Buns
found id 4753 for Large-Batch Hot Dog Buns
working on Lefse
found id 5733 for Lefse
working on Lemon Brioche
found id 392 for Lemon Brioche
working on Lemon Buns
found id 7293 for Lemon Buns
working on Lemon-Cherry Buns
found id 7955 for Lemon-Cherry Buns
working on Lemon Love Buns
found id 342 for Lemon Love Buns
working on Lemon Poppy Seed Swirl Bread
found id 9423 for Lemon Poppy Seed Swirl Bread
working on Lemon Sour Cream Bread
found id 8087 for Lemon Sour Cream Bread
working on Lemon Sugar Crunch Buns
found id 4017 for Lemon Sugar Crunch Buns
working on Light-As-Air Seed Bread
found id 1390 for Light-As-Air Seed Bread
working on Light Summer Ciabatta
found id 6967 for Light Summer Ciabatta
workin

found id 7233 for Onion Hamburger Rolls
working on Onion Parmesan Cracker Bread
found id 5001 for Onion Parmesan Cracker Bread
working on Onion Rolls
found id 8247 for Onion Rolls
working on Orange Blossoms
found id 7627 for Orange Blossoms
working on Orange-Scented Cinnamon Rolls
found id 8773 for Orange-Scented Cinnamon Rolls
working on Orange Sunshine Coffeecake
found id 5829 for Orange Sunshine Coffeecake
working on Orange Sweet Rolls
found id 4413 for Orange Sweet Rolls
working on Organic Wheat Pita with Fruit Salsa
found id 7931 for Organic Wheat Pita with Fruit Salsa
working on Our Best-Ever Oatmeal Bread
found id 7127 for Our Best-Ever Oatmeal Bread
working on Our Easiest Stollen
found id 1308 for Our Easiest Stollen
working on Our Favorite Bread By Hand and By Machine
found id 6727 for Our Favorite Bread By Hand and By Machine
working on Our Favorite Sandwich Bread
found id 6693 for Our Favorite Sandwich Bread
working on Our Favorite Sticky Buns
found id 6249 for Our Favorite 

found id 4035 for Rieska
working on Roasted Apple Bread
found id 3907 for Roasted Apple Bread
working on Roasted Garlic Potato Bread
found id 8269 for Roasted Garlic Potato Bread
working on Roasted Hazelnut and Raisin Flutes
found id 8273 for Roasted Hazelnut and Raisin Flutes
working on Roasted Strawberry Cream Cheese Rolls
found id 9591 for Roasted Strawberry Cream Cheese Rolls
working on Roasted Vegetable Focaccia
found id 4425 for Roasted Vegetable Focaccia
working on Robin's Ethereal Air Bread
found id 7667 for Robin's Ethereal Air Bread
working on Robyn's Jalapeño Cheese Sour Cream Bread
found id 7693 for Robyn's Jalapeño Cheese Sour Cream Bread
working on Rosemary, Asiago & Olive Bread
found id 1666 for Rosemary, Asiago & Olive Bread
working on Rosemary Cheddar Cheese Bread
found id 6825 for Rosemary Cheddar Cheese Bread
working on Rosemary Potato Rolls
found id 7553 for Rosemary Potato Rolls
working on Rum-Raisin Bread
found id 6145 for Rum-Raisin Bread
working on Russian Black

found id 9231 for Sourdough Pumpkin Spice Bread
working on Sourdough Rye Bread
found id 6657 for Sourdough Rye Bread
working on Sourdough Sandwich Bread
found id 8969 for Sourdough Sandwich Bread
working on Sourdough Starter
found id 5759 for Sourdough Starter
working on Sparkling Harvest Crisps
found id 1400 for Sparkling Harvest Crisps
working on Spelt Biscuits
found id 8027 for Spelt Biscuits
working on Spelt Pita
found id 9666 for Spelt Pita
working on Spiced Banana Bread
found id 9435 for Spiced Banana Bread
working on Spiced Cherry Breakfast Ring
found id 9141 for Spiced Cherry Breakfast Ring
working on Spicy Onion Pinwheels
found id 8735 for Spicy Onion Pinwheels
working on Spicy Spiral Bread
found id 8259 for Spicy Spiral Bread
working on Spicy Tomato-Filled Tunisian Flatbreads
found id 9144 for Spicy Tomato-Filled Tunisian Flatbreads
working on Spinach-Filled Lebanese Flatbreads
found id 9030 for Spinach-Filled Lebanese Flatbreads
working on Spiraled Wheat Loaf
found id 8435 f

found id 6281 for Walter Sands' Favorite Bread Machine Bread
working on Warm-Your-Heart Buns
found id 8727 for Warm-Your-Heart Buns
working on Water Bagels
found id 6521 for Water Bagels
working on Westphalian Rye Bread
found id 7269 for Westphalian Rye Bread
working on Wheat-Oat-Flax Buns
found id 2079 for Wheat-Oat-Flax Buns
working on White Breakfast Bread
found id 6295 for White Breakfast Bread
working on White Chili with Cornbread
found id 1843 for White Chili with Cornbread
working on White Chocolate Bread
found id 8007 for White Chocolate Bread
working on White Chocolate-Cinnamon Bread
found id 8315 for White Chocolate-Cinnamon Bread
working on White Chocolate Cinnamon Rolls
found id 4857 for White Chocolate Cinnamon Rolls
working on White Sandwich Bread
found id 26 for White Sandwich Bread
working on White Wheat Pretzels
found id 8009 for White Wheat Pretzels
working on Whole-Grain Banana Bread
found id 4527 for Whole-Grain Banana Bread
working on Whole Grain Pain au Chocolat
f

In [134]:
with open('recipe_list.pkl', 'wb') as f:
    pickle.dump(file=f, obj=recipes)

In [135]:
def get_printable_url(rid):
    return f'https://www.kingarthurflour.com/recipes/PrintRecipe?RID={rid}&radio=3'


In [136]:
len(recipes)

881

In [205]:
def get_recipe_text(recipe: Recipe):
    xs = get_search_results(get_printable_url(recipe.rid))
    description = xs.find('div', {'id': 'HeadNotes'}).text
    for li in xs.find('div', {'id':'g_ingredients'}).find_all('li'):
        li.insert(0, '\n')   # Careful, this mutates xs
    ingreds = xs.find('div', {'id': 'g_ingredients'}).text
    directions = xs.find('div', {'class': 'recipe-instructions'}).text
    try:
        tips = xs.find('div', {'id': 'helpful-tips'}).text
    except AttributeError:
        tips = ''
    recipe.text = {}
    recipe.text['description'] = description
    recipe.text['ingreds'] = ingreds
    recipe.text['directions'] = directions
    recipe.text['tips'] = tips
    recipe.text_raw = recipe.title + '\n' + description + '\n' + ingreds + directions + tips

In [206]:
for r in recipes:
    print(f'working on {r.title}')
    get_recipe_text(r)
    time.sleep(0.25)  # Don't dos them

working on 9-Grain Bread


/Users/andrewho/.pyenv/versions/breadventure/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/andrewho/.pyenv/versions/3.6.5/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


working on 10-Grain Orange Bread
working on 100% Whole Wheat Banana Bread
working on 100% Whole Wheat Bread for the Bread Machine
working on 100% Whole Wheat Cinnamon Swirl Bread
working on 100% Whole Wheat Pain de Mie
working on 100% Whole Wheat Sandwich Bread
working on 100% Whole Wheat Zucchini Chocolate Chip Bread
working on A Dozen Simple Bagels
working on A Simple, Rustic Loaf
working on A Smaller 100% Whole Wheat Pain de Mie
working on A Smaller Pain de Mie
working on Absolutely No-Knead Crusty-Chewy Bread
working on Alexandra's Crescents
working on All-American Ham and Cheese Roll
working on Almond Cracked Wheat Bread
working on Almond Flour Zucchini Bread
working on Almond Sweet Bread Kit
working on American Irish Soda Bread
working on American-Style Panettone
working on Amish Dinner Rolls
working on Ancient Grains Bread
working on Angel Biscuits
working on Anise-Chocolate Flatbread
working on Apple Cheddar Chop Bread
working on Apple Cheese Loaf
working on Apple-Cinnamon Brea

working on Double Chocolate Banana Bread
working on Double Chocolate Zucchini Bread
working on Double Light Rye Bread
working on Dried Fruit Focaccia
working on Dutch Oven Dinner Rolls
working on Easter Monday Ham & Cheese Bake
working on Easy Breadstick Twists
working on Easy Chocolate Pudding Cake
working on Easy Cinnamon Bread
working on Easy Crusty Baguettes
working on Easy-Does-It Cinnamon Buns
working on Easy Focaccia For Panini
working on Easy Gluten-Free Pumpkin Bread made with baking mix
working on Easy Hot Cross Buns
working on Easy Jelly Doughnut Holes
working on Easy Pumpkin Bread
working on Easy Whole Grain Pumpkin-Banana Bread
working on Easy Whole Wheat Apple-Raisin Bread
working on Eggnog French Toast
working on Eggplant Parmesan Calzones
working on English Muffin Bread for the Bread Machine
working on English Muffin Toasting Bread
working on English Muffins
working on European-Style Hearth Bread
working on Everyday Whole Grain Bread
working on Everything-Cheddar Filled

working on Mini Picnic Rolls
working on Mission Fig Bread
working on Molasses Multi-Seed Bread
working on Molly's Challah
working on Mona's Braided Onion Loaf
working on Monkey in a Tree
working on Monkeying Around Bread
working on Moravian Fruit Coffeecake
working on Multi-Grain Molasses Bread
working on Multi-Seed Braid
working on Multi-Seed Hamburger Buns
working on Multigrain Brown Bread
working on Multigrain Rolls
working on Multigrain Sourdough Boule
working on Multigrain Sourdough Sandwich Bread
working on Multigrain Sunflower Bread
working on Mustard Rye Sandwich Bread
working on Naturally Leavened Brioche-Style Kugelhopf
working on Naturally Leavened Sourdough Bread
working on New England Anadama Bread
working on New England Hot Dog Buns
working on New England Thanksgiving Bread
working on New Orleans Pecan Pie Bread Pudding
working on No-Knead 100% Whole Wheat Bread
working on No-Knead Amaranth Honey-Nut Bread
working on No-Knead Balsamic-Glazed Cherry Focaccia
working on No-

working on Sharon's Whole Grain Dinner Rolls & Bread
working on Shortcut Russenzopf
working on Shredded Wheat Bread
working on Simple Tortillas
working on Simple Zucchini Bread
working on Six-Grain Baguettes
working on Six-Grain Bread
working on Skillet Cornbread
working on Skillet Flatbreads
working on Slovak Paska
working on Slow Cooker Boston Brown Bread
working on Smith Family Pasta Bake
working on Smokehouse Cranberry Cheese Bread
working on Smoky Fougasse
working on Soft Bread Sticks
working on Soft Cinnamon Rolls
working on Soft Cinnamon Twists
working on Soft Dinner Rolls
working on Soft Garlic Knots
working on Soft Onion Rolls
working on Soft Pretzels
working on Soft Sandwich Bread with Flax
working on Soft Sandwich Rolls
working on Soft White Dinner Rolls
working on Soft Wrap Bread
working on Sopa Paraguaya
working on Sopa Paraguaya (Paraguayan Cornbread)
working on Sour Cream Apricot Coffee Bread
working on Sour Cream & Chive Potato Bread or Rolls
working on Sour Cream Rye M

In [207]:
# Save again
with open('recipes_complete.pkl', 'wb') as f:
    pickle.dump(file=f, obj=recipes)

In [208]:
data_dir = 'data/recipes/individual/'